In [2]:
# MYSQL Connection string

config = {
  'user': 'root',
  'password': 'Nls@123',
  'host': '127.0.0.1',
  'database': 'nlscore_sidian',
  'auth_plugin' : 'mysql_native_password',
  'raise_on_warnings': True
}

In [3]:
# num_days = calendar.monthrange(int(YEAR), int(MONTH))[1]
# START_DATE = YEAR + '-' + MONTH + '-' + '01'
# END_DATE = YEAR + '-' + MONTH + '-' + str(num_days)

In [4]:
# selecting the transaction code that should be charged
# connection establish
mysql_connection = mysql.connector.connect(**config)
# data access using pandas
TRANSACTION_CHARGE_CODES = pd.read_sql('select * from transaction$charge$codes where TRANSACTION_CODE = '+TRAN_CODE+'', con=mysql_connection)
# connection close
mysql_connection.close()
TRANSACTION_CHARGE_CODES.head()

,ID,TRANSACTION_TYPE,TRANSACTION_NAME,TRANSACTION_CODE,CHARGE_TRANSACTION_CODE,TAX_TRANSACTION_CODE,CHARGE_CODE,TAX,IS_ALWAYS_REGENERATE
0,6,IB,IB_PESALINK-70,738,234,465,KITSCOMM,20,0
1,7,IB,IB_PESALINK-70,738,51,None,PESALINKCHG,None,0


In [5]:
# list which includes the transaction code,charge code,tax code
TRANSACTION_CODE = TRANSACTION_CHARGE_CODES['TRANSACTION_CODE'].unique().tolist()
# CHARGE_CODE = [TRANSACTION_CHARGE_CODES['CHARGE_TRANSACTION_CODE'][0]]
CHARGE_CODE = TRANSACTION_CHARGE_CODES['CHARGE_TRANSACTION_CODE'].unique().tolist()
TAX_CODE = TRANSACTION_CHARGE_CODES['TAX_TRANSACTION_CODE'].unique().tolist()

TRANSACTION_CODE = [x for x in TRANSACTION_CODE if x is not None]
CHARGE_CODE = [x for x in CHARGE_CODE if x is not None]
TAX_CODE = [x for x in TAX_CODE if x is not None]

print('TRANSACTION_CODE - ',TRANSACTION_CODE)
print('CHARGE_CODE - ',CHARGE_CODE)
print('TAX_CODE - ',TAX_CODE)

TRANSACTION_CODE -  ['738']
CHARGE_CODE -  ['234', '51']
TAX_CODE -  ['465']


In [6]:
# converting codes to list
CODES = TRANSACTION_CODE + CHARGE_CODE + TAX_CODE
print('CODES - ',CODES)

CODES -  ['738', '234', '51', '465']


In [7]:
# converting list to string
FILTER_CODES = ','.join(CODES)

FILTER_CODES

'738,234,51,465'

In [8]:
# from dateutil.relativedelta import relativedelta
# START_DATE1 = '2022-02-01'
# END_DATE1 = '2022-02-05'

# date_list = [START_DATE1]
# # pd.date_range(START_DATE1, END_DATE1, freq='M').tolist()
# cur_date = datetime.strptime(START_DATE1, '%Y-%m-%d').date()
# end = datetime.strptime(END_DATE1, '%Y-%m-%d').date()

# while cur_date < end:
#     cur_date += relativedelta(months=1)
#     if (cur_date > end):
#         cur_date = end
#     date_list.append(str(cur_date))
    
# date_list

In [9]:
# # selecting the data using filter codes
# for i in range(0,len(date_list)):
#     if(len(date_list) == 1):
#         START_DATE = date_list[i]
#         END_DATE = date_list[i]
#     elif(i == (len(date_list)-1)):
#             break
#     else:
#         START_DATE = date_list[i]
#         END_DATE = date_list[i + 1]
        
#     mysql_connection = mysql.connector.connect(**config)
#     df = pd.read_sql('SELECT AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP from statement$entry_2022_jan12 where cast(TIME_STAMP as date) between "'+str(START_DATE)+'" and "'+str(END_DATE)+'" and TRANSACTION_CODE in ('+FILTER_CODES+')', con=mysql_connection)
#     mysql_connection.close()
#     print ('START_DATE - ',START_DATE)
#     print ('END_DATE - ',END_DATE)
#     print ('LENGTH - ',len(df))
#     print("Hiii")

In [10]:
# mysql_connection = mysql.connector.connect(**config)
# dfa = pd.read_sql('SELECT TIME_STAMP from daywise$transaction$analysis where cast(TIME_STAMP as date) between "'+str(START_DATE)+'" and "'+str(END_DATE)+'" and TRANSACTION_CODE = '+TRAN_CODE+'', con=mysql_connection)
# mysql_connection.close()
# dfa.head()

In [11]:
# # converting time_stamp as list
# dfa['TIME_STAMP'] = pd.to_datetime(dfa['TIME_STAMP'])
# dates = dfa['TIME_STAMP'].dt.strftime('%Y-%m-%d').unique().tolist()
# dates

In [12]:
# # timestamp string to datetime conversion
# df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'])
# # converting date_time column to date column to filter
# df['DATE'] = df['TIME_STAMP'].dt.date
# df['DATE'] = pd.to_datetime(df['DATE'])
# # filtering dates which has the history records
# df = df[~df['DATE'].isin(dates)]
# df.reset_index(drop=True, inplace=True)
# print("LEN - ", len(df))
# df.head()

In [13]:
# selecting the data using filter codes
mysql_connection = mysql.connector.connect(**config)
df = pd.read_sql('SELECT AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP from statement$entry_2022_jan12 where cast(TIME_STAMP as date) between "'+str(START_DATE)+'" and "'+str(END_DATE)+'" and TRANSACTION_CODE in ('+FILTER_CODES+')', con=mysql_connection)
mysql_connection.close()
df.tail()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP
1329656,955.20,51,FTN2013002953YC\HOF,2022-01-13 03:02:34
1329657,2497.45,51,FTN2013003136BC,2022-01-13 03:02:34
1329658,2497.45,51,FTN2013003136BC\HOF,2022-01-13 03:02:34
1329659,248.75,51,FTN2013004203UL,2022-01-13 03:13:10
1329660,248.75,51,FTN2013004203UL\HOF,2022-01-13 03:13:15


In [14]:
# dataframe info
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329661 entries, 0 to 1329660
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   AMOUNT_LCY        1329661 non-null  float64       
 1   TRANSACTION_CODE  1329661 non-null  object        
 2   TRANS_REFERENCE   1329661 non-null  object        
 3   TIME_STAMP        1329661 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 188.6 MB


In [15]:
# removing annotations from trans reference
df['TRANS_REFERENCE'] = df['TRANS_REFERENCE'].str.split("\\").str[0]

In [16]:
# filtering transaction charge code data's for trans reference
TRANS_REFERENCE = df[df['TRANSACTION_CODE'].isin(TRANSACTION_CODE)]
TRANS_REFERENCE = TRANS_REFERENCE[['TRANS_REFERENCE']]
TRANS_REFERENCE.reset_index(drop=True, inplace=True)

In [17]:
# converting trans reference as list
TRANS_REFERENCE = TRANS_REFERENCE['TRANS_REFERENCE'].unique().tolist()

In [18]:
# filtering charge code data's using trans reference
df = df[df['TRANS_REFERENCE'].isin(TRANS_REFERENCE)]
df.reset_index(drop=True, inplace=True)
df.head()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP
0,-83.6,234,FTN1365221627DR,2022-01-01 01:35:57
1,-27500.0,738,FTN1365221627DR,2022-01-01 01:35:57
2,12.0,465,FTN1365221627DR,2022-01-01 01:35:57
3,27500.0,51,FTN1365221627DR,2022-01-01 01:35:57
4,11.6,51,FTN1365221627DR,2022-01-01 01:36:00


In [19]:
# total no of unique chragable transations
print("Total TRANS_REFERENCE count - ",len(df['TRANS_REFERENCE'].unique().tolist()))

Total TRANS_REFERENCE count -  2363


In [20]:
# creating list for map the code with respective transactions
TRANSACTION_CODES = [TRANSACTION_CODE,CHARGE_CODE,TAX_CODE]
print('TRANSACTION_CODES -',TRANSACTION_CODES)
TRANSACTION_TYPES = ['TRANSACTION_AMOUNT','CHARGE_AMOUNT','TAX_AMOUNT']
print('TRANSACTION_TYPES -',TRANSACTION_TYPES)

TRANSACTION_CODES - [['738'], ['234', '51'], ['465']]
TRANSACTION_TYPES - ['TRANSACTION_AMOUNT', 'CHARGE_AMOUNT', 'TAX_AMOUNT']


In [21]:
# json data creation for mapping code with respective transactions
data = {}
for i in range(0,len(TRANSACTION_TYPES)):
    for j in range(0,len(TRANSACTION_CODES[i])):
        data[TRANSACTION_CODES[i][j]] = TRANSACTION_TYPES[i]
data

{'738': 'TRANSACTION_AMOUNT',
 '234': 'CHARGE_AMOUNT',
 '51': 'CHARGE_AMOUNT',
 '465': 'TAX_AMOUNT'}

In [22]:
# Creating Transaction type column
df['TRANSACTION_TYPE'] = df['TRANSACTION_CODE'].map(data)
df.head()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP,TRANSACTION_TYPE
0,-83.6,234,FTN1365221627DR,2022-01-01 01:35:57,CHARGE_AMOUNT
1,-27500.0,738,FTN1365221627DR,2022-01-01 01:35:57,TRANSACTION_AMOUNT
2,12.0,465,FTN1365221627DR,2022-01-01 01:35:57,TAX_AMOUNT
3,27500.0,51,FTN1365221627DR,2022-01-01 01:35:57,CHARGE_AMOUNT
4,11.6,51,FTN1365221627DR,2022-01-01 01:36:00,CHARGE_AMOUNT


In [23]:
df.head(200).to_csv('sample.csv')

In [23]:
# pivoting the table using TRANS_REFERENCE
df_pivot = df.pivot_table(index=['TRANS_REFERENCE'], columns='TRANSACTION_TYPE', values='AMOUNT_LCY').reset_index()
df_pivot.fillna(0, inplace=True)
df_pivot.tail()

TRANSACTION_TYPE,TRANS_REFERENCE,CHARGE_AMOUNT,TAX_AMOUNT,TRANSACTION_AMOUNT
2358,FTN220112IXTKBL,7992.72,8.0,-20000.0
2359,FTN220112IXTKBM,8987.92,12.0,-22500.0
2360,FTN220112IXTKBN,4992.72,8.0,-12500.0
2361,FTN220112IXTKBO,5992.72,8.0,-15000.0
2362,FTN220112IXTKEK,163963.92,32.0,-410000.0


In [24]:
# merging 2 dataframes
df = pd.merge(df, df_pivot, on=['TRANS_REFERENCE'])
# filtering duplicates by keeping trans reference
df = df[df['TRANSACTION_TYPE'].isin(['TRANSACTION_AMOUNT'])]
df.reset_index(drop=True, inplace=True)
# dropping duplicate values
df = df.drop_duplicates(["TRANS_REFERENCE"],ignore_index = True)
# # converting transaction amount negative sign to positive sign
df['TRANSACTION_AMOUNT'] = df['TRANSACTION_AMOUNT'].abs()
df.head()

,AMOUNT_LCY,TRANSACTION_CODE,TRANS_REFERENCE,TIME_STAMP,TRANSACTION_TYPE,CHARGE_AMOUNT,TAX_AMOUNT,TRANSACTION_AMOUNT
0,-27500.0,738,FTN1365221627DR,2022-01-01 01:35:57,TRANSACTION_AMOUNT,10987.92,12.0,27500.0
1,-5000.0,738,FTN1365214111SS,2022-01-01 01:09:19,TRANSACTION_AMOUNT,1995.12,6.0,5000.0
2,-245000.0,738,FTN1365232946SM,2022-01-01 02:32:04,TRANSACTION_AMOUNT,97963.92,32.0,245000.0
3,-144500.0,738,FTN1365211429IS,2022-01-01 00:36:48,TRANSACTION_AMOUNT,57773.52,24.0,144500.0
4,-17000.0,738,FTN1365211918AU,2022-01-01 00:39:30,TRANSACTION_AMOUNT,6792.72,8.0,17000.0


In [25]:
df = df[['TRANS_REFERENCE','TRANSACTION_CODE','TRANSACTION_AMOUNT','CHARGE_AMOUNT','TAX_AMOUNT','TIME_STAMP']]
df.tail()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP
2358,FTN2012162851GW,738,360000.0,143963.92,32.0,2022-01-12 19:01:03
2359,FTN2012173203JJ,738,84755.0,33885.12,16.0,2022-01-12 20:02:42
2360,FTN2012190329QJ,738,100000.0,39983.12,16.0,2022-01-12 21:34:16
2361,FTN2012205826WY,738,240000.0,95963.92,32.0,2022-01-12 23:29:59
2362,FTN2012212958JM,738,170000.0,67973.52,24.0,2022-01-13 00:02:42


In [26]:
# re validating total no of unique chragable transations
print("Total TRANS_REFERENCE count - ",len(df['TRANS_REFERENCE'].unique().tolist()))

Total TRANS_REFERENCE count -  2363


In [27]:
# for i in range(0,len(df)):
#     if(df['CHARGE_AMOUNT'][i] != 0):
#         df['CHARGE_AMOUNT'][i] = df['CHARGE_AMOUNT'][i] + df['TAX_AMOUNT'][i]
#     else:
#         df['CHARGE_AMOUNT'][i] = df['CHARGE_AMOUNT'][i]

In [28]:
df.loc[(df['CHARGE_AMOUNT'] != 0),'CHARGE_AMOUNT'] = df[['CHARGE_AMOUNT','TAX_AMOUNT']].sum(axis=1)
df.loc[(df['CHARGE_AMOUNT'] == 0), 'CHARGE_AMOUNT'] = df['CHARGE_AMOUNT']
# # converting charge amount negative sign to positive sign
df['CHARGE_AMOUNT'] = df['CHARGE_AMOUNT'].abs()
df.head()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP
0,FTN1365221627DR,738,27500.0,10999.92,12.0,2022-01-01 01:35:57
1,FTN1365214111SS,738,5000.0,2001.12,6.0,2022-01-01 01:09:19
2,FTN1365232946SM,738,245000.0,97995.92,32.0,2022-01-01 02:32:04
3,FTN1365211429IS,738,144500.0,57797.52,24.0,2022-01-01 00:36:48
4,FTN1365211918AU,738,17000.0,6800.72,8.0,2022-01-01 00:39:30


In [30]:
# checking the history records
def mysql_readTable(query,param=None):
    
    connection = mysql_connection
    dataframe = pd.read_sql(query, con=connection,params=param)
    
    return dataframe

In [31]:
mysql_connection = mysql.connector.connect(**config)
param = {
            'TRANSACTION_CODE':TRAN_CODE
        }
query = "select CHARGE_CODE,TAX from transaction$charge$codes where TRANSACTION_CODE = %(TRANSACTION_CODE)s"
TRAN_CHARGE_CODES = mysql_readTable(query,param)
TRAN_CHARGE_CODES.head()

,CHARGE_CODE,TAX
0,KITSCOMM,20
1,PESALINKCHG,None


In [32]:
CHARGE_CODE_id = TRAN_CHARGE_CODES['CHARGE_CODE'].unique().tolist()
FILTER_CODES = '|'.join(CHARGE_CODE_id)
print(FILTER_CODES)
param = {
            'CRG_CODE': FILTER_CODES
        }

KITSCOMM|PESALINKCHG


In [32]:
# selecting the history table using charge code
query_his = "select id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME from bnk_charge$types$history where id REGEXP %(CRG_CODE)s"
CHARGE_HIS = mysql_readTable(query_his,param)
is_charge_history = len(CHARGE_HIS)
CHARGE_HIS.head()

,id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME
0,ACTOMP;1,60.00,None,None,None,None,None,2011-06-01 16:55:00.0
1,ACTOMP;2,75.00,None,None,None,None,None,2022-01-01 12:39:00.0
2,EFTSTF;1,100.00,None,None,None,None,None,2011-06-01 16:55:00.0


In [33]:
if(is_charge_history != 0):
    
    # selecting charge table 
    query = "select id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME from bnk_charge$types where id REGEXP %(CRG_CODE)s"
    df_charges = mysql_readTable(query,param)

    # merging chare table with charge history table
    df_charges = pd.concat([df_charges,CHARGE_HIS], ignore_index=True)
    df_charges = df_charges.sort_values(by='DATE_TIME', ascending=True)
    df_charges.reset_index(drop=True, inplace=True)
    df_charges['TRANSACTION_CODE'] = TRAN_CODE
    df_charges['TAX'] = TRAN_CHARGE_CODES['TAX'][0]
else:
    # selecting the transaction code charges
    mysql_connection = mysql.connector.connect(**config)
    #df_charges = pd.read_sql('select tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MAXIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
    df_charges = pd.read_sql('select bct.id,tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MINIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE =  '+TRAN_CODE+' ', con=mysql_connection)
    mysql_connection.close()

df_charges.head()

,id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME,TRANSACTION_CODE,TAX
0,ACTOMP;1,60.00,None,None,None,None,None,2011-06-01 16:55:00.0,186,20
1,EFTSTF;1,100.00,None,None,None,None,None,2011-06-01 16:55:00.0,186,20
2,ACTOMP;2,75.00,None,None,None,None,None,2022-01-01 12:39:00.0,186,20
3,ACTOMP,80.00,None,None,None,None,None,None,186,20
4,ACTOMP1,None,LEVEL^LEVEL^LEVEL^LEVEL^LEVEL^LEVEL,0^0^0^0^0^0,20000.00^30000.00^40000.00^50000.00^60000.00,75.00^145.00^170.00^215.00^245.00^265.00,None,None,186,20


In [34]:
# for charge history duration mapping
def fn_history_charges_duration(df_charges):
    df_charges['START_DATE'] = np.nan
    df_charges['END_DATE'] = np.nan
    for i in range (0,len(df_charges)):
        if (i ==0):
            df_charges['END_DATE'][i] = df_charges['DATE_TIME'][i]
        elif (df_charges['DATE_TIME'][i] == None):
            df_charges['START_DATE'][i] = str(pd.to_datetime(df_charges['DATE_TIME']).max())
        else:
            df_charges['START_DATE'][i] = df_charges['DATE_TIME'][i - 1]
            df_charges['END_DATE'][i] = df_charges['DATE_TIME'][i]
    df_charges = df_charges[['id','FLAT_AMOUNT','CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','TRANSACTION_CODE','TAX','START_DATE','END_DATE']]
    # datatype conversion
    df_charges['START_DATE'] = pd.to_datetime(df_charges['START_DATE'])
    df_charges['END_DATE'] = pd.to_datetime(df_charges['END_DATE'])
    
    return df_charges

In [35]:
# for first char validation. remove '^' if first charecter without any values
def fn_level_band_firstchar_validation(df_charges):
    for column in df_charges:
        for i in range(0,len(df_charges)):
            if(df_charges[column][i] != None):
                if(str(df_charges[column][i][0]) == '^'):
                    df_charges[column][i] =  df_charges[column][i][1:]
    return df_charges

In [36]:
def fn_str_split(string):
    return string.split('^')

In [37]:
def fn_level_band_split(df_charges,is_history):
    
    cols_to_df_list = ['CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','id','TAX']
    df_chg = pd.DataFrame(columns = cols_to_df_list)
    
    # split multiple level/band into multiple rows
    for j in range (0,len(df_charges)):
        df_len = len(df_chg)
        for i in cols_to_df_list:
            if df_charges[i][j]:
                split_list = fn_str_split(df_charges[i][j])
                for k in range (0,len(split_list)):
                    if((i == "id") | (i == 'TAX')):
                        df_chg.loc[df_len + k,i] = df_charges[i][j]
                    else:
                        df_chg.loc[df_len + k,i] = split_list[k]
    
    # fill the empty rows using forward fill
    df_chg['id'] = df_chg[['id']].ffill()
    df_chg['TAX'] = df_chg.groupby('id')['TAX'].ffill()
    df_chg=df_chg.replace(r'', 0, regex=True)
    
    # data type conversion
    df_chg['PERCENTAGE'] = df_chg['PERCENTAGE'].astype('float')
    df_chg['MAX_AMT'] = df_chg['UPTO_AMOUNT'].astype('float')
    df_chg['MIN_CHARGE'] = df_chg['MINIMUM_AMOUNT'].astype('float')
    df_chg['MAX_CHARGE'] = df_chg['MAXIMUM_AMOUNT'].astype('float')
    df_chg['TAX'] = df_chg['TAX'].astype('float')
    
    # creating the minimum amount for the level/band
    df_chg['MIN_AMT'] = 0
    for i in range(0,len(df_chg)):
        if(i == 0):
            df_chg['MIN_AMT'][i] =  0
        else:
            df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1

    df_chg.loc[(df_chg['MIN_AMT'].isnull()) | (df_chg['MIN_AMT'] > df_chg['MAX_AMT']),'MIN_AMT'] = 0
    df_chg = df_chg[["id","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE","TAX"]]
    
    if(is_history != 0):
        df_chg['START_DATE'] = df_chg.id.map(df_charges.set_index('id')['START_DATE'])
        df_chg['END_DATE'] = df_chg.id.map(df_charges.set_index('id')['END_DATE'])
    
    return df_chg

In [38]:
# For FLAT AMOUNT with no history
def fn_flat_amount_with_no_history(df,df_charges):
    data = {}
    for i in range(0,len(df_charges)):
        # Creating expected charge column
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['FLAT_AMOUNT'][i]
        df['EXPECTED_CHARGE'] = df['TRANSACTION_CODE'].map(data)
        # Creating expected Tax percentage
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['TAX'][i]
        df['EXPECTED_TAX'] = df['TRANSACTION_CODE'].map(data)
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        # expected tax calculation
        df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
        
        return df

In [39]:
# for single level/band with no history 
def fn_single_level_band_with_no_history(df,df_charges):
    data = {}
    for i in range(0,len(df_charges)): 
        # Creating expected charge percentage
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['PERCENTAGE'][i]
        df['EXPECTED_CHARGE'] = df['TRANSACTION_CODE'].map(data)
        # Creating expected Tax percentage
        data[df_charges['TRANSACTION_CODE'][i]] = df_charges['TAX'][i]
        df['EXPECTED_TAX'] = df['TRANSACTION_CODE'].map(data)
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        df_charges['MINIMUM_AMOUNT'] = df_charges['MINIMUM_AMOUNT'].astype('float')
        df_charges['MAXIMUM_AMOUNT'] = df_charges['MAXIMUM_AMOUNT'].astype('float')
        # expected charge and tax calculation
        df['EXPECTED_CHARGE'] = round(df['TRANSACTION_AMOUNT'] /100 * df['EXPECTED_CHARGE'],2)
        df['EXPECTED_TAX'] = round(df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX'],2)
        # comparing charges with minimum maximum amount
        # for charge lesser then minimum charge amount
        if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
            df.loc[(df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
        # for charge greter then minimum charge amount
        if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
            df.loc[(df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]      
    
    return df

In [40]:
# for multi level with no history 
def fn_multi_level_with_no_history(df,df_chg):
    # for expected charge initialization
    df['EXPECTED_CHARGE'] = 0
    df['EXPECTED_TAX'] = 0

    for i in range (0,len(df_chg)):
        # for charge as per percentage
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
        # for charge lesser then minimum charge amount
        if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
        # for charge greter then minimum charge amount
        if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
        # for tax as per percentage
        if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

        # for expected charge
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
        # for expected tax
        if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

    df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
    df_chg.reset_index(drop=True, inplace=True)

    for i in range (0,len(df_chg)):
        # for charge as per percentage
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
        # for charge lesser then minimum charge amount
        if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
        # for charge greter then minimum charge amount
        if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
        # for tax as per percentage
        if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

        # for expected charge
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
        # for expected tax
        if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  
        
    return df

In [41]:
# for multi band with no history 
def fn_multi_band_with_no_history(df,df_chg):
    # for expected charge initialization
    df['EXPECTED_CHARGE'] = 0
    df['EXPECTED_TAX'] = 0

    for i in range (0,len(df_chg)):
        # Chargable amount calculation
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
        if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
            df.loc[(df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
        else:
            df.loc[(df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
        # for charge as per percentage
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
        # for tax as per percentage
        if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

        # for expected charge
        df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
        # for expected tax
        if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
            df.loc[(df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
            
    return df

In [42]:
# For FLAT AMOUNT with history
def fn_flat_amount_with_history(df,df_charges):
    for i in range(0,len(df_charges)):
        # Creating expected charge and expected tax percentage column
        if(pd.isnull(df_charges['START_DATE'][i])):
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        elif(pd.isnull(df_charges['END_DATE'][i])):
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        else:
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        # expected tax calculation
        df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
        
    return df

In [43]:
# for single level/band with history 
def fn_single_level_band_with_history(df,df_charges):
    for i in range(0,len(df_charges)): 
        if(pd.isnull(df_charges['START_DATE'][i])):
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
            df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        elif(pd.isnull(df_charges['END_DATE'][i])):
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
        else:
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
            df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i] 
        #conversion
        df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
        df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
        df_charges['MINIMUM_AMOUNT'] = df_charges['MINIMUM_AMOUNT'].astype('float')
        df_charges['MAXIMUM_AMOUNT'] = df_charges['MAXIMUM_AMOUNT'].astype('float')
        # expected charge and tax calculation
        df['EXPECTED_CHARGE'] = round(df['TRANSACTION_AMOUNT'] /100 * df['EXPECTED_CHARGE'],2)
        df['EXPECTED_TAX'] = round(df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX'],2)
        # comparing charges with minimum maximum amount
        if(pd.isnull(df_charges['START_DATE'][i])):
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
        elif(pd.isnull(df_charges['END_DATE'][i])):
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
        else:
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
                
    return df

In [44]:
# for multi level with history 
def fn_multi_level_with_history(df,df_chg):
    # for expected charge initialization
    df['EXPECTED_CHARGE'] = 0
    df['EXPECTED_TAX'] = 0

    for i in range (0,len(df_chg)):
        if(pd.isnull(df_chg['START_DATE'][i])):
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)


        elif(pd.isnull(df_chg['END_DATE'][i])):  
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax & 
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)



        else:
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)



    df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
    df_chg.reset_index(drop=True, inplace=True)

    for i in range (0,len(df_chg)):
        if(pd.isnull(df_chg['START_DATE'][i])):
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  


        elif(pd.isnull(df_chg['END_DATE'][i])):
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  


        else:
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for charge lesser then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
            # for charge greter then minimum charge amount
            if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  
                
    return df

In [45]:
# for multi band with history 
def fn_multi_band_with_history(df,df_chg):
    # for expected charge initialization
    df['EXPECTED_CHARGE'] = 0
    df['EXPECTED_TAX'] = 0

    for i in range (0,len(df_chg)):
        if(pd.isnull(df_chg['START_DATE'][i])):
            # Chargable amount calculation
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
            else:
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)


        elif(pd.isnull(df_chg['END_DATE'][i])):
            # Chargable amount calculation
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
            else:
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)


        else:
            # Chargable amount calculation
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
            else:
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
            # for charge as per percentage
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
            # for tax as per percentage
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

            # for expected charge
            df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
            # for expected tax
            if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
                df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
    return df

In [46]:
if((is_charge_history == 0) & (df_charges['FLAT_AMOUNT'][0] != None)):
    df = fn_flat_amount_with_no_history(df,df_charges)
elif((is_charge_history == 0) & ((df_charges['CALC_TYPE'][0] == 'LEVEL') | (df_charges['CALC_TYPE'][0] == 'BAND'))):
    df_charges = fn_level_band_firstchar_validation(df_charges)
    df = fn_single_level_band_with_no_history(df,df_charges)
elif((is_charge_history == 0) & (df_charges['FLAT_AMOUNT'][0] == None)): 
    df_charges = fn_level_band_firstchar_validation(df_charges)
    df_chg = fn_level_band_split(df_charges,is_charge_history)
    if(df_chg['CALC_TYPE'][0] == "LEVEL"):
        df = fn_multi_level_with_no_history(df,df_chg)
    elif(df_chg['CALC_TYPE'][0] == "BAND"): 
        df = fn_multi_band_with_no_history(df,df_chg)
elif((is_charge_history != 0) & (df_charges['FLAT_AMOUNT'][0] != None)):
    df_charges = fn_history_charges_duration(df_charges)
    df = fn_flat_amount_with_history(df,df_charges)
elif((is_charge_history == 0) & ((df_charges['CALC_TYPE'][0] == 'LEVEL') | (df_charges['CALC_TYPE'][0] == 'BAND'))):
    df_charges = fn_level_band_firstchar_validation(df_charges)
    df_charges = fn_history_charges_duration(df_charges)
    df = fn_single_level_band_with_history(df,df_charges)
elif((is_charge_history != 0) & (df_charges['FLAT_AMOUNT'][0] == None)): 
    df_charges = fn_level_band_firstchar_validation(df_charges)
    df_charges = fn_history_charges_duration(df_charges)
    df_chg = fn_level_band_split(df_charges,is_charge_history)
    if(df_chg['CALC_TYPE'][0] == "LEVEL"):
        df = fn_multi_level_with_history(df,df_chg)
    elif(df_chg['CALC_TYPE'][0] == "BAND"): 
        df = fn_multi_band_with_history(df,df_chg)

/tmp/ipykernel_18664/1308296552.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_charges['END_DATE'][i] = df_charges['DATE_TIME'][i]
/home/sabarish/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_18664/1308296552.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_charges['START_DATE'][i] = df_charges

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TRANS_REFERENCE     2308 non-null   object        
 1   TRANSACTION_CODE    2308 non-null   object        
 2   TRANSACTION_AMOUNT  2308 non-null   float64       
 3   CHARGE_AMOUNT       2308 non-null   float64       
 4   TAX_AMOUNT          2308 non-null   float64       
 5   TIME_STAMP          2308 non-null   datetime64[ns]
 6   EXPECTED_CHARGE     2308 non-null   float64       
 7   EXPECTED_TAX        2308 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 144.4+ KB


In [48]:
# # json data creation for transaction codes with respective Tax
# data = {}
# for i in range(0,len(df_charges)):
#     if(df_charges['FLAT_AMOUNT'][i] != None):
#         data[df_charges['TRANSACTION_CODE'][i]] = df_charges['TAX'][i]
# data

In [49]:
# # Creating expected Tax column
# df['EXPECTED_TAX'] = df['TRANSACTION_CODE'].map(data)
# df.head()

In [50]:
# data type conversion
df['CHARGE_AMOUNT'] = df['CHARGE_AMOUNT'].astype('float')
df['TAX_AMOUNT'] = df['TAX_AMOUNT'].astype('float')
# df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
# df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')

In [51]:
# # actual tax calculation
# df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
# df.head()

In [52]:
# charge_deviation calculation
df['CHARGE_DEVIATION'] = df['CHARGE_AMOUNT'] - df['EXPECTED_CHARGE']
# tax_deviation calculation
df['TAX_DEVIATION'] = df['TAX_AMOUNT'] - df['EXPECTED_TAX']
# # overall deviation
# df['OVERALL_DEVIATION'] = df['CHARGE_DEVIATION'] + df['TAX_DEVIATION']
df.head()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP,EXPECTED_CHARGE,EXPECTED_TAX,CHARGE_DEVIATION,TAX_DEVIATION
0,FTN1365215258WF,186,21500.0,100.0,20.0,2022-01-01 01:12:27,75.0,4.746094,25.0,15.253906
1,FTN2001050254ZY,186,500000.0,100.0,20.0,2022-01-01 07:39:48,75.0,4.746094,25.0,15.253906
2,FTN2001192218JJ,186,100000.0,100.0,20.0,2022-01-01 22:09:17,100.0,20.000000,0.0,0.000000
3,FTN2001095714QQ,186,200000.0,100.0,20.0,2022-01-01 12:28:49,75.0,4.746094,25.0,15.253906
4,FTN2001124050AM,186,10000.0,0.0,0.0,2022-01-01 15:23:12,100.0,20.000000,-100.0,-20.000000


In [53]:
# precisely_chrgd calculation
df.loc[(df['CHARGE_DEVIATION'] == 0) & (df['TAX_DEVIATION'] == 0),'PRECISELY_CHARGED'] = True
df.loc[(df['CHARGE_DEVIATION'] != 0) | (df['TAX_DEVIATION'] != 0),'PRECISELY_CHARGED'] = False
df.to_csv('final.csv')
df.head()

,TRANS_REFERENCE,TRANSACTION_CODE,TRANSACTION_AMOUNT,CHARGE_AMOUNT,TAX_AMOUNT,TIME_STAMP,EXPECTED_CHARGE,EXPECTED_TAX,CHARGE_DEVIATION,TAX_DEVIATION,PRECISELY_CHARGED
0,FTN1365215258WF,186,21500.0,100.0,20.0,2022-01-01 01:12:27,75.0,4.746094,25.0,15.253906,False
1,FTN2001050254ZY,186,500000.0,100.0,20.0,2022-01-01 07:39:48,75.0,4.746094,25.0,15.253906,False
2,FTN2001192218JJ,186,100000.0,100.0,20.0,2022-01-01 22:09:17,100.0,20.000000,0.0,0.000000,True
3,FTN2001095714QQ,186,200000.0,100.0,20.0,2022-01-01 12:28:49,75.0,4.746094,25.0,15.253906,False
4,FTN2001124050AM,186,10000.0,0.0,0.0,2022-01-01 15:23:12,100.0,20.000000,-100.0,-20.000000,False


In [73]:
# converting Time_stamp to yyyy-mm-dd format
dates = df['TIME_STAMP'].dt.strftime('%Y-%m-%d').unique().tolist()
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'].dt.strftime('%Y-%m-%d'))

In [74]:
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'].dt.strftime('%Y-%m-%d'))

EXPECTED_COUNT = len(df.loc[df['TIME_STAMP'] == dates[1]].TRANS_REFERENCE.unique())
print('EXPECTED_COUNT - ',EXPECTED_COUNT)

PRECISELY_CHARGED = df.loc[(df['TIME_STAMP'] == dates[1]) & (df['PRECISELY_CHARGED'] == True)].PRECISELY_CHARGED.count()
print('PRECISELY_CHARGED - ',PRECISELY_CHARGED)

NOT_PRECISELY_CHARGED = df.loc[(df['TIME_STAMP'] == dates[1]) & (df['PRECISELY_CHARGED'] == False)].PRECISELY_CHARGED.count()
print('NOT_PRECISELY_CHARGED - ',NOT_PRECISELY_CHARGED)

MISSING_CHARGE = df.loc[(df['TIME_STAMP'] == dates[1]) & (df['CHARGE_AMOUNT'] == 0)].CHARGE_AMOUNT.count()
print('MISSING_CHARGE - ',MISSING_CHARGE)

MISSING_TAX_BUT_CHARGE_PRESENT = df.loc[(df['TIME_STAMP'] == dates[1]) & (df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].CHARGE_AMOUNT.count()
print('MISSING_TAX_BUT_CHARGE_PRESENT - ',MISSING_TAX_BUT_CHARGE_PRESENT)

WRONG_CHARGE = df.loc[(df['TIME_STAMP'] == dates[0]) &(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].CHARGE_AMOUNT.count()
print('WRONG_CHARGE - ',WRONG_CHARGE)

EXPECTED_COUNT -  1533
PRECISELY_CHARGED -  24
NOT_PRECISELY_CHARGED -  1509
MISSING_CHARGE -  1509
MISSING_TAX_BUT_CHARGE_PRESENT -  0
WRONG_CHARGE -  3


In [78]:
EXPECTED_CHARGE_AMOUNT = df.loc[df['TIME_STAMP'] == dates[0]].EXPECTED_CHARGE.sum()
print('EXPECTED_CHARGE_AMOUNT - ',EXPECTED_CHARGE_AMOUNT)

EXPECTED_TAX_AMOUNT = df.loc[df['TIME_STAMP'] == dates[0]].EXPECTED_TAX.sum()
print('EXPECTED_TAX_AMOUNT - ',EXPECTED_TAX_AMOUNT)

ACTUAL_CHARGE_AMOUNT = df.loc[df['TIME_STAMP'] == dates[0]].CHARGE_AMOUNT.sum()
print('ACTUAL_CHARGE_AMOUNT - ',ACTUAL_CHARGE_AMOUNT)

ACTUAL_TAX_AMOUNT = df.loc[df['TIME_STAMP'] == dates[0]].TAX_AMOUNT.sum()
print('ACTUAL_TAX_AMOUNT - ',ACTUAL_TAX_AMOUNT)

REVENUE_LOSS_IN_CHARGE_AMOUNT = df.loc[(df['TIME_STAMP'] == dates[0]) & (df['CHARGE_DEVIATION'] < 0)].CHARGE_DEVIATION.sum()
print("REVENUE_LOSS_IN_CHARGE_AMOUNT -",REVENUE_LOSS_IN_CHARGE_AMOUNT)

REVENUE_EXTRA_IN_CHARGE_AMOUNT = df.loc[(df['TIME_STAMP'] == dates[0]) & (df['CHARGE_DEVIATION'] > 0)].CHARGE_DEVIATION.sum()
print("REVENUE_EXTRA_IN_CHARGE_AMOUNT -",REVENUE_EXTRA_IN_CHARGE_AMOUNT)

REVENUE_LOSS_IN_TAX_AMOUNT = df.loc[(df['TIME_STAMP'] == dates[0]) & (df['TAX_DEVIATION'] < 0)].TAX_DEVIATION.sum()
print("REVENUE_LOSS_IN_TAX_AMOUNT -",REVENUE_LOSS_IN_TAX_AMOUNT)

REVENUE_EXTRA_IN_TAX_AMOUNT =df.loc[(df['TIME_STAMP'] == dates[0]) & (df['TAX_DEVIATION'] > 0)].TAX_DEVIATION.sum()
print("REVENUE_EXTRA_IN_TAX_AMOUNT -",REVENUE_EXTRA_IN_TAX_AMOUNT)

REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT = ACTUAL_CHARGE_AMOUNT - EXPECTED_CHARGE_AMOUNT
print("REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT -",REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT)

REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT = ACTUAL_TAX_AMOUNT - EXPECTED_TAX_AMOUNT
print("REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT -",REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT)

REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT = df.loc[(df['TIME_STAMP'] == dates[0]) & (df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].CHARGE_DEVIATION.sum()
print('REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT - ',REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT)

REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT = df.loc[(df['TIME_STAMP'] == dates[0]) & (df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].CHARGE_DEVIATION.sum()
print('REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT - ',REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT)

EXPECTED_CHARGE_AMOUNT -  525.0
EXPECTED_TAX_AMOUNT -  74.23828125
ACTUAL_CHARGE_AMOUNT -  400.0
ACTUAL_TAX_AMOUNT -  80.0
REVENUE_LOSS_IN_CHARGE_AMOUNT - -200.0
REVENUE_EXTRA_IN_CHARGE_AMOUNT - 75.0
REVENUE_LOSS_IN_TAX_AMOUNT - -40.0
REVENUE_EXTRA_IN_TAX_AMOUNT - 45.76171875
REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT - -125.0
REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT - 5.76171875
REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT -  0.0
REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT -  75.0


In [57]:
CREATED_BY = 'ML_USER'
CREATED_ON = datetime.now()

split= "^"
daywise_list = []
for i in range(0,len(dates)):
    EXPECTED_CHARGES = split.join(df.loc[df['TIME_STAMP'] == dates[i]].TRANS_REFERENCE.unique().tolist())
    PRECISELY_CHARGED = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['PRECISELY_CHARGED'] == True)].TRANS_REFERENCE.tolist())
    NOT_PRECISELY_CHARGED = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['PRECISELY_CHARGED'] == False)].TRANS_REFERENCE.tolist())
    MISSING_CHARGE = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] == 0)].TRANS_REFERENCE.tolist())
    MISSING_TAX_BUT_CHARGE_PRESENT = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].TRANS_REFERENCE.tolist())
    WRONG_CHARGE = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) &(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].TRANS_REFERENCE.tolist())
    #REVENUE_LOSS_EXTRA_IN_CHARGE = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_DEVIATION'] != 0)].TRANS_REFERENCE.tolist())
    REVENUE_LOSS_IN_CHARGE_AMOUNT = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_DEVIATION'] < 0)].TRANS_REFERENCE.tolist())
    REVENUE_EXTRA_IN_CHARGE_AMOUNT = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_DEVIATION'] > 0)].TRANS_REFERENCE.tolist())
    #REVENUE_LOSS_EXTRA_IN_TAX = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['TAX_DEVIATION'] == 0)].TRANS_REFERENCE.tolist())
    REVENUE_LOSS_IN_TAX_AMOUNT = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['TAX_DEVIATION'] < 0)].TRANS_REFERENCE.tolist())
    REVENUE_EXTRA_IN_TAX_AMOUNT = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['TAX_DEVIATION'] > 0)].TRANS_REFERENCE.tolist())
    REVENUE_BELOW_ACCEPTANCE_IN_CHARGE = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].TRANS_REFERENCE.tolist())
    REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE = split.join(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].TRANS_REFERENCE.tolist())
    
    day_list = [TRAN_CODE,dates[i],EXPECTED_CHARGES,PRECISELY_CHARGED,NOT_PRECISELY_CHARGED,MISSING_CHARGE,
                 MISSING_TAX_BUT_CHARGE_PRESENT,WRONG_CHARGE,REVENUE_LOSS_EXTRA_IN_CHARGE,
                 REVENUE_LOSS_EXTRA_IN_TAX,REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,
                 REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,CREATED_BY,CREATED_ON]

    daywise_list.append(day_list)

In [58]:
dfs = pd.DataFrame(daywise_list)
dfs.columns = ['TRANSACTION_CODE','TIME_STAMP','EXPECTED_CHARGES','PRECISELY_CHARGED','NOT_PRECISELY_CHARGED',
               'MISSING_CHARGE','MISSING_TAX_BUT_CHARGE_PRESENT','WRONG_CHARGE','REVENUE_LOSS_EXTRA_IN_CHARGE',
               'REVENUE_LOSS_EXTRA_IN_TAX','REVENUE_BELOW_ACCEPTANCE_IN_CHARGE','REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE',
                'CREATED_BY','CREATED_ON']
dfs['TIME_STAMP'] = pd.to_datetime(dfs['TIME_STAMP'])
dfs.head()

In [59]:
# pip install SQLAlchemy
# pip install mysqlclient
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

DB = URL(drivername='mysql', host='127.0.0.1',port='3306',username='root',password='Nls@123',database='nlscore_sidian')
engine = create_engine(name_or_url=DB)

mysql_connection = engine.connect()

dfs.to_sql(con=mysql_connection, name='daywise$trans_reference$details', if_exists='append',index=False)
mysql_connection.close()

In [60]:
mysql_connection = mysql.connector.connect(**config)
dfs = pd.read_sql('select * from daywise$trans_reference$details where TIME_STAMP between "'+str(START_DATE)+'" and "'+str(END_DATE)+'" and TRANSACTION_CODE = "'+TRAN_CODE+'"', con=mysql_connection)
mysql_connection.close()
dfs = dfs.drop_duplicates(["TIME_STAMP"],ignore_index = True)
dfs.head()

In [61]:
EXPECTED_CHARGES = split.join(dfs['EXPECTED_CHARGES'].loc[dfs['EXPECTED_CHARGES'] != ''].tolist())
PRECISELY_CHARGED = split.join(dfs['PRECISELY_CHARGED'].loc[dfs['PRECISELY_CHARGED'] != ''].tolist())
NOT_PRECISELY_CHARGED = split.join(dfs['NOT_PRECISELY_CHARGED'].loc[dfs['NOT_PRECISELY_CHARGED'] != ''].tolist())
MISSING_CHARGE = split.join(dfs['MISSING_CHARGE'].loc[dfs['MISSING_CHARGE'] != ''].tolist())
MISSING_TAX_BUT_CHARGE_PRESENT = split.join(dfs['MISSING_TAX_BUT_CHARGE_PRESENT'].loc[dfs['MISSING_TAX_BUT_CHARGE_PRESENT'] != ''].tolist())
WRONG_CHARGE = split.join(dfs['WRONG_CHARGE'].loc[dfs['WRONG_CHARGE'] != ''].tolist())
REVENUE_LOSS_EXTRA_IN_CHARGE = split.join(dfs['REVENUE_LOSS_EXTRA_IN_CHARGE'].loc[dfs['REVENUE_LOSS_EXTRA_IN_CHARGE'] != ''].tolist())
REVENUE_LOSS_EXTRA_IN_TAX = split.join(dfs['REVENUE_LOSS_EXTRA_IN_TAX'].loc[dfs['REVENUE_LOSS_EXTRA_IN_TAX'] != ''].tolist())
REVENUE_BELOW_ACCEPTANCE_IN_CHARGE = split.join(dfs['REVENUE_BELOW_ACCEPTANCE_IN_CHARGE'].loc[dfs['REVENUE_BELOW_ACCEPTANCE_IN_CHARGE'] != ''].tolist())
REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE = split.join(dfs['REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE'].loc[dfs['REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE'] != ''].tolist())

In [62]:
mysql_connection = mysql.connector.connect(**config)
def mysql_insertData(query,data):
    
    connection = mysql_connection
    cursor = connection.cursor()
    cursor.execute(query, data)
    connection.commit()
    primary_key = cursor.lastrowid
    cursor.close()
    connection.close()
    
    return primary_key

In [63]:
TRANSACTION_TYPE = 'MOBILE-EFT-91'
TRANSACTION_CODE = '186'

param = {
            'TRANSACTION_TYPE' : TRANSACTION_TYPE,
            'TRANSACTION_CODE' : TRANSACTION_CODE,
            'START_DATE' : START_DATE,
            'END_DATE' : END_DATE,
            'EXPECTED_CHARGES' : EXPECTED_CHARGES,
            'PRECISELY_CHARGED' : PRECISELY_CHARGED,
            'NOT_PRECISELY_CHARGED' : NOT_PRECISELY_CHARGED,
            'MISSING_CHARGE' : MISSING_CHARGE,
            'MISSING_TAX_BUT_CHARGE_PRESENT' : MISSING_TAX_BUT_CHARGE_PRESENT,
            'WRONG_CHARGE' : WRONG_CHARGE,
            'REVENUE_LOSS_EXTRA_IN_CHARGE' : REVENUE_LOSS_EXTRA_IN_CHARGE,
            'REVENUE_LOSS_EXTRA_IN_TAX' : REVENUE_LOSS_EXTRA_IN_TAX,
            'REVENUE_BELOW_ACCEPTANCE_IN_CHARGE' : REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,
            'REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE' : REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,
            "CREATED_BY":'ML_USER',
            "CREATED_ON":datetime.now()
            
}

MYSQL_INSERT_TRANS_REFERENCE_DETAILS = ("INSERT INTO transaction$trans_reference$details"
                                        "(TRANSACTION_TYPE,TRANSACTION_CODE,START_DATE,END_DATE,EXPECTED_CHARGES,PRECISELY_CHARGED,NOT_PRECISELY_CHARGED,MISSING_CHARGE,MISSING_TAX_BUT_CHARGE_PRESENT,WRONG_CHARGE,REVENUE_LOSS_EXTRA_IN_CHARGE,REVENUE_LOSS_EXTRA_IN_TAX,REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,CREATED_BY,CREATED_ON)"
                                        "VALUES (%(TRANSACTION_TYPE)s, %(TRANSACTION_CODE)s,%(START_DATE)s,%(END_DATE)s, %(EXPECTED_CHARGES)s,%(PRECISELY_CHARGED)s,%(NOT_PRECISELY_CHARGED)s,%(MISSING_CHARGE)s,%(MISSING_TAX_BUT_CHARGE_PRESENT)s,%(WRONG_CHARGE)s,%(REVENUE_LOSS_EXTRA_IN_CHARGE)s,%(REVENUE_LOSS_EXTRA_IN_TAX)s,%(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE)s,%(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE)s,%(CREATED_BY)s,%(CREATED_ON)s)")

mysql_insertData(MYSQL_INSERT_TRANS_REFERENCE_DETAILS,param)

In [64]:
#TRANS_REFERENCE DAY WISE SPLIT/INSERT and MERGE to START,END date CODE END

In [65]:
# daywise_list = []
# for i in range(0,len(dates)):
    
#     EXPECTED_COUNT = len(df.loc[df['TIME_STAMP'] == dates[i]].TRANS_REFERENCE.unique())
#     PRECISELY_CHARGED = df.loc[(df['TIME_STAMP'] == dates[i]) & (df['PRECISELY_CHARGED'] == True)].PRECISELY_CHARGED.count()
#     NOT_PRECISELY_CHARGED = df.loc[(df['TIME_STAMP'] == dates[i]) & (df['PRECISELY_CHARGED'] == False)].PRECISELY_CHARGED.count()
#     MISSING_CHARGE = df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] == 0)].CHARGE_AMOUNT.count()
#     MISSING_TAX_BUT_CHARGE_PRESENT = df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].CHARGE_AMOUNT.count()
#     WRONG_CHARGE = df.loc[(df['TIME_STAMP'] == dates[i]) &(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].CHARGE_AMOUNT.count()
    
#     EXPECTED_CHARGE_AMOUNT = round(df.loc[df['TIME_STAMP'] == dates[i]].EXPECTED_CHARGE.sum(),2)
#     EXPECTED_TAX_AMOUNT = round(df.loc[df['TIME_STAMP'] == dates[i]].EXPECTED_TAX.sum(),2)
#     ACTUAL_CHARGE_AMOUNT = round(df.loc[df['TIME_STAMP'] == dates[i]].CHARGE_AMOUNT.sum(),2)
#     ACTUAL_TAX_AMOUNT = round(df.loc[df['TIME_STAMP'] == dates[i]].TAX_AMOUNT.sum(),2)
#     REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT = round(ACTUAL_CHARGE_AMOUNT - EXPECTED_CHARGE_AMOUNT,2)
#     REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT = round(ACTUAL_TAX_AMOUNT - EXPECTED_TAX_AMOUNT,2)
#     REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT = round(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].CHARGE_DEVIATION.sum(),2)
#     REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT = round(df.loc[(df['TIME_STAMP'] == dates[i]) & (df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].CHARGE_DEVIATION.sum(),2)
    
#     loop_list = [TRAN_CODE,dates[i],EXPECTED_COUNT,PRECISELY_CHARGED,NOT_PRECISELY_CHARGED,MISSING_CHARGE,
#                  MISSING_TAX_BUT_CHARGE_PRESENT,WRONG_CHARGE,EXPECTED_CHARGE_AMOUNT,EXPECTED_TAX_AMOUNT,
#                  ACTUAL_CHARGE_AMOUNT,ACTUAL_TAX_AMOUNT,REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT,
#                  REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT,REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT,
#                  REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT,CREATED_BY,CREATED_ON]
#     daywise_list.append(loop_list)

In [66]:
# dfs = pd.DataFrame(daywise_list)
# dfs.columns = ['TRANSACTION_CODE','TIME_STAMP','EXPECTED_COUNT','PRECISELY_CHARGED','NOT_PRECISELY_CHARGED',
#                'MISSING_CHARGE','MISSING_TAX_BUT_CHARGE_PRESENT','WRONG_CHARGE','EXPECTED_CHARGE_AMOUNT',
#                'EXPECTED_TAX_AMOUNT','ACTUAL_CHARGE_AMOUNT','ACTUAL_TAX_AMOUNT','REVENUE_LOSS_EXTRA_IN_CHARGE',
#                'REVENUE_LOSS_EXTRA_IN_TAX','REVENUE_BELOW_ACCEPTANCE_IN_CHARGE','REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE',
#                 'CREATED_BY','CREATED_ON']
# dfs['TIME_STAMP'] = pd.to_datetime(dfs['TIME_STAMP'])
# dfs.head()

In [67]:
# mysql_connection = engine.connect()

# dfs.to_sql(con=mysql_connection, name='daywise$transaction$analysis', if_exists='append',index=False)
# mysql_connection.close()

In [68]:
# mysql_connection = mysql.connector.connect(**config)
# dfs = pd.read_sql('select * from daywise$transaction$analysis where TIME_STAMP between "'+str(START_DATE)+'" and "'+str(END_DATE)+'" and TRANSACTION_CODE = "'+TRAN_CODE+'"', con=mysql_connection)
# mysql_connection.close()
# dfs = dfs.drop_duplicates(["TIME_STAMP"],ignore_index = True)
# dfs.head()

In [69]:
# EXPECTED_COUNT = dfs['EXPECTED_COUNT'].sum()
# PRECISELY_CHARGED = dfs['PRECISELY_CHARGED'].sum()
# NOT_PRECISELY_CHARGED = dfs['NOT_PRECISELY_CHARGED'].sum()
# MISSING_CHARGE = dfs['MISSING_CHARGE'].sum()
# MISSING_TAX_BUT_CHARGE_PRESENT = dfs['MISSING_TAX_BUT_CHARGE_PRESENT'].sum()
# WRONG_CHARGE = dfs['WRONG_CHARGE'].sum()

In [70]:
# EXPECTED_CHARGE_AMOUNT = dfs['EXPECTED_CHARGE_AMOUNT'].sum()
# EXPECTED_TAX_AMOUNT = dfs['EXPECTED_TAX_AMOUNT'].sum()
# ACTUAL_CHARGE_AMOUNT = dfs['ACTUAL_CHARGE_AMOUNT'].sum()
# ACTUAL_TAX_AMOUNT = dfs['ACTUAL_TAX_AMOUNT'].sum()
# REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT = dfs['REVENUE_LOSS_EXTRA_IN_CHARGE'].sum()
# REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT = dfs['REVENUE_LOSS_EXTRA_IN_TAX'].sum()
# REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT = dfs['REVENUE_BELOW_ACCEPTANCE_IN_CHARGE'].sum()
# REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT = dfs['REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE'].sum()

In [71]:
# print ("EXPECTED CHARGES COUNT - ", EXPECTED_COUNT)
# print ("PRECISELY CHARGED COUNT - ", PRECISELY_CHARGED)
# print ("NOT PRECISELY CHARGED COUNT - ", NOT_PRECISELY_CHARGED)
# # SUBSET ONE
# print("MISSING CHARGE COUNT - ",MISSING_CHARGE)
# print("MISSING TAX BUT CHARGE PRESENT COUNT - ",MISSING_TAX_BUT_CHARGE_PRESENT)
# # SUBSET TWO
# print("WRONG CHARGE COUNT- ",WRONG_CHARGE)

In [72]:
# print("EXPECTED CHARGE AMOUNT - ",round(EXPECTED_CHARGE_AMOUNT,2))
# print("EXPECTED TAX AMOUNT - ",round(EXPECTED_TAX_AMOUNT,2))
# print("ACTUAL CHARGE AMOUNT - ",round(ACTUAL_CHARGE_AMOUNT,2))
# print("ACTUAL TAX AMOUNT - ",round(ACTUAL_TAX_AMOUNT,2))
# print("REVENUE LOSS/EXTRA IN CHARGE AMOUNT - ",round(REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT,2))
# print("REVENUE LOSS/EXTRA IN TAX AMOUNT - ",round(REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT,2))
# print("REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT - ",round(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT,2))
# print("REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT - ",round(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT,2))

In [68]:
#DAY WISE  REVENUE ANALYSIS SPLIT/INSERT and MERGE to START,END date CODE END

In [69]:
EXPECTED_COUNT = len(df['TRANS_REFERENCE'].unique().tolist())
PRECISELY_CHARGED = df['PRECISELY_CHARGED'].loc[df['PRECISELY_CHARGED'] == True].count()
NOT_PRECISELY_CHARGED = df['PRECISELY_CHARGED'].loc[df['PRECISELY_CHARGED'] == False].count()
MISSING_CHARGE = df['CHARGE_AMOUNT'].loc[df['CHARGE_AMOUNT'] == 0].count()
MISSING_TAX_BUT_CHARGE_PRESENT = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].CHARGE_AMOUNT.count()
WRONG_CHARGE = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].CHARGE_AMOUNT.count()

In [70]:
EXPECTED_CHARGE_AMOUNT = df['EXPECTED_CHARGE'].sum()
EXPECTED_TAX_AMOUNT = df['EXPECTED_TAX'].sum()
ACTUAL_CHARGE_AMOUNT = df['CHARGE_AMOUNT'].sum()
ACTUAL_TAX_AMOUNT = df['TAX_AMOUNT'].sum()
REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT = ACTUAL_CHARGE_AMOUNT - EXPECTED_CHARGE_AMOUNT
REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT = ACTUAL_TAX_AMOUNT - EXPECTED_TAX_AMOUNT
REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].CHARGE_DEVIATION.sum()
REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT = df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].CHARGE_DEVIATION.sum()

In [71]:
print ("EXPECTED CHARGES COUNT - ", EXPECTED_COUNT)
print ("PRECISELY CHARGED COUNT - ", PRECISELY_CHARGED)
print ("NOT PRECISELY CHARGED COUNT - ", NOT_PRECISELY_CHARGED)
# SUBSET ONE
print("MISSING CHARGE COUNT - ",MISSING_CHARGE)
print("MISSING TAX BUT CHARGE PRESENT COUNT - ",MISSING_TAX_BUT_CHARGE_PRESENT)
# SUBSET TWO
print("WRONG CHARGE COUNT- ",WRONG_CHARGE)

EXPECTED CHARGES COUNT -  2308
PRECISELY CHARGED COUNT -  298
NOT PRECISELY CHARGED COUNT -  2010
MISSING CHARGE COUNT -  2007
MISSING TAX BUT CHARGE PRESENT COUNT -  0
WRONG CHARGE COUNT-  3


In [72]:
print ("PRECISELY CHARGED PERCENTAGE - " , round(PRECISELY_CHARGED / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("NOT PRECISELY CHARGED PERCENTAGE - " , round(NOT_PRECISELY_CHARGED / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("MISSING CHARGE PERCENTAGE - " , round(MISSING_CHARGE / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("MISSING TAX BUT CHARGE PRESENT PERCENTAGE - " , round(MISSING_TAX_BUT_CHARGE_PRESENT / EXPECTED_COUNT * 100,2).astype(str)+'%')
print ("WRONG CHARGE PERCENTAGE - " , round(WRONG_CHARGE / EXPECTED_COUNT * 100,2).astype(str)+'%')

PRECISELY CHARGED PERCENTAGE -  12.91%
NOT PRECISELY CHARGED PERCENTAGE -  87.09%
MISSING CHARGE PERCENTAGE -  86.96%
MISSING TAX BUT CHARGE PRESENT PERCENTAGE -  0.0%
WRONG CHARGE PERCENTAGE -  0.13%


In [73]:
print("EXPECTED CHARGE AMOUNT - ",round(EXPECTED_CHARGE_AMOUNT,2))
print("EXPECTED TAX AMOUNT - ",round(EXPECTED_TAX_AMOUNT,2))
print("ACTUAL CHARGE AMOUNT - ",round(ACTUAL_CHARGE_AMOUNT,2))
print("ACTUAL TAX AMOUNT - ",round(ACTUAL_TAX_AMOUNT,2))
print("REVENUE LOSS/EXTRA IN CHARGE AMOUNT - ",round(REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT,2))
print("REVENUE LOSS/EXTRA IN TAX AMOUNT - ",round(REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT,2))
print("REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT - ",round(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT,2))
print("REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT - ",round(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT,2))

EXPECTED CHARGE AMOUNT -  230725.0
EXPECTED TAX AMOUNT -  46114.24
ACTUAL CHARGE AMOUNT -  30100.0
ACTUAL TAX AMOUNT -  6060.0
REVENUE LOSS/EXTRA IN CHARGE AMOUNT -  -200625.0
REVENUE LOSS/EXTRA IN TAX AMOUNT -  -40054.24
REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT -  0.0
REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT -  75.0


In [74]:
print("ACTUAL CHARGE AMOUNT PERCENTAGE - ",round(ACTUAL_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')
print("ACTUAL TAX AMOUNT PERCENTAGE - ",round(ACTUAL_TAX_AMOUNT/EXPECTED_TAX_AMOUNT*100 ,2).astype(str)+'%')
print("REVENUE LOSS/EXTRA IN CHARGE AMOUNT PERCENTAGE - ",round(REVENUE_LOSS_OR_EXTRA_IN_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')
print("REVENUE LOSS/EXTRA IN TAX AMOUNT PERCENTAGE - ",round(REVENUE_LOSS_OR_EXTRA_IN_TAX_AMOUNT/EXPECTED_TAX_AMOUNT*100,2).astype(str)+'%')
print("REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE - ",round(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')
print("REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE - ",round(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE_AMOUNT/EXPECTED_CHARGE_AMOUNT*100,2).astype(str)+'%')

ACTUAL CHARGE AMOUNT PERCENTAGE -  13.05%
ACTUAL TAX AMOUNT PERCENTAGE -  13.14%
REVENUE LOSS/EXTRA IN CHARGE AMOUNT PERCENTAGE -  -86.95%
REVENUE LOSS/EXTRA IN TAX AMOUNT PERCENTAGE -  -86.86%
REVENUE BELOW ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE -  0.0%
REVENUE ABOVE ACCEPTANCE IN CHARGE AMOUNT PERCENTAGE -  0.03%


In [75]:
split= "^"

EXPECTED_CHARGES = split.join(df['TRANS_REFERENCE'].unique().tolist())
PRECISELY_CHARGED = split.join(df.loc[df['PRECISELY_CHARGED'] == True].TRANS_REFERENCE.tolist())
NOT_PRECISELY_CHARGED = split.join(df.loc[df['PRECISELY_CHARGED'] == False].TRANS_REFERENCE.tolist())
MISSING_CHARGE = split.join(df.loc[df['CHARGE_AMOUNT'] == 0].TRANS_REFERENCE.tolist())
MISSING_TAX_BUT_CHARGE_PRESENT = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['TAX_AMOUNT'] == 0)].TRANS_REFERENCE.tolist())
WRONG_CHARGE = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] != 0)].TRANS_REFERENCE.tolist())
REVENUE_LOSS_EXTRA_IN_CHARGE = split.join(df.loc[df['CHARGE_DEVIATION'] != 0].TRANS_REFERENCE.tolist())
REVENUE_LOSS_EXTRA_IN_TAX = split.join(df.loc[df['TAX_DEVIATION'] == 0].TRANS_REFERENCE.tolist())
REVENUE_BELOW_ACCEPTANCE_IN_CHARGE = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] < 0) ==True].TRANS_REFERENCE.tolist())
REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE = split.join(df.loc[(df['CHARGE_AMOUNT'] != 0) & (df['CHARGE_DEVIATION'] > 0) ==True].TRANS_REFERENCE.tolist())

In [76]:
mysql_connection = mysql.connector.connect(**config)
def mysql_insertData(query,data):
    
    connection = mysql_connection
    cursor = connection.cursor()
    cursor.execute(query, data)
    connection.commit()
    primary_key = cursor.lastrowid
    cursor.close()
    connection.close()
    
    return primary_key

In [77]:
TRANSACTION_TYPE = 'MOBILE-EFT-91'
TRANSACTION_CODE = '186'

param = {
            'TRANSACTION_TYPE' : TRANSACTION_TYPE,
            'TRANSACTION_CODE' : TRANSACTION_CODE,
            'START_DATE' : START_DATE,
            'END_DATE' : END_DATE,
            'EXPECTED_CHARGES' : EXPECTED_CHARGES,
            'PRECISELY_CHARGED' : PRECISELY_CHARGED,
            'NOT_PRECISELY_CHARGED' : NOT_PRECISELY_CHARGED,
            'MISSING_CHARGE' : MISSING_CHARGE,
            'MISSING_TAX_BUT_CHARGE_PRESENT' : MISSING_TAX_BUT_CHARGE_PRESENT,
            'WRONG_CHARGE' : WRONG_CHARGE,
            'REVENUE_LOSS_EXTRA_IN_CHARGE' : REVENUE_LOSS_EXTRA_IN_CHARGE,
            'REVENUE_LOSS_EXTRA_IN_TAX' : REVENUE_LOSS_EXTRA_IN_TAX,
            'REVENUE_BELOW_ACCEPTANCE_IN_CHARGE' : REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,
            'REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE' : REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,
            "CREATED_BY":'ML_USER',
            "CREATED_ON":datetime.now()
            
}

MYSQL_INSERT_TRANS_REFERENCE_DETAILS = ("INSERT INTO transaction$trans_reference$details"
                                        "(TRANSACTION_TYPE,TRANSACTION_CODE,START_DATE,END_DATE,EXPECTED_CHARGES,PRECISELY_CHARGED,NOT_PRECISELY_CHARGED,MISSING_CHARGE,MISSING_TAX_BUT_CHARGE_PRESENT,WRONG_CHARGE,REVENUE_LOSS_EXTRA_IN_CHARGE,REVENUE_LOSS_EXTRA_IN_TAX,REVENUE_BELOW_ACCEPTANCE_IN_CHARGE,REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE,CREATED_BY,CREATED_ON)"
                                        "VALUES (%(TRANSACTION_TYPE)s, %(TRANSACTION_CODE)s,%(START_DATE)s,%(END_DATE)s, %(EXPECTED_CHARGES)s,%(PRECISELY_CHARGED)s,%(NOT_PRECISELY_CHARGED)s,%(MISSING_CHARGE)s,%(MISSING_TAX_BUT_CHARGE_PRESENT)s,%(WRONG_CHARGE)s,%(REVENUE_LOSS_EXTRA_IN_CHARGE)s,%(REVENUE_LOSS_EXTRA_IN_TAX)s,%(REVENUE_BELOW_ACCEPTANCE_IN_CHARGE)s,%(REVENUE_ABOVE_ACCEPTANCE_IN_CHARGE)s,%(CREATED_BY)s,%(CREATED_ON)s)")

mysql_insertData(MYSQL_INSERT_TRANS_REFERENCE_DETAILS,param)

6

In [11]:
def mysql_connection():
    
    mysql_connection = mysql.connector.connect(**config)
    return mysql_connection


def mysql_deleteData(query,data):
    
    connection = mysql_connection()
    cursor = connection.cursor()
    cursor.execute(query, data)
    connection.commit()
    cursor.close()
    connection.close()

In [18]:
CURRENT_DATE = datetime.now().strftime('%Y-%m-%d')
CURRENT_DATE

'2022-02-15'

In [15]:
param = {
            "CURRENT_DATE":"2022-01-11"
        }
query = "delete from daywise$transaction$analysis where TIME_STAMP = %(CURRENT_DATE)s"
query1 = "delete from daywise$trans_reference$details where TIME_STAMP = %(CURRENT_DATE)s"

mysql_deleteData(query,param)
mysql_deleteData(query1,param)

In [78]:
# def mysql_readTable(query,param=None):
    
#     connection = mysql_connection
#     dataframe = pd.read_sql(query, con=connection,params=param)
    
#     return dataframe

In [79]:
# import calendar

# num_days = calendar.monthrange(int(YEAR), int(MONTH))[1]
# START_DATE = YEAR + '-' + MONTH + '-' + '01'
# END_DATE = YEAR + '-' + MONTH + '-' + str(num_days)

In [80]:
# mysql_connection = mysql.connector.connect(**config)
# param = {
#             'TRAN_CODE':TRAN_CODE
#         }
# query = "select CHARGE_CODE,TAX from transaction$charge$codes where TRANSACTION_CODE = %(TRAN_CODE)s"
# TRAN_CHARGE_CODES = mysql_readTable(query,param)
# TRAN_CHARGE_CODES.head()
# # TRAN_CHARGE_HISTORY['TRANSACTION_CODE'] = TRAN_CHARGE_HISTORY_1['TRANSACTION_CODE'][0]
# # TRAN_CHARGE_HISTORY['TAX'] = TRAN_CHARGE_HISTORY_1['TAX'][0]
# # TRAN_CHARGE_HISTORY.head()

In [81]:
# CHARGE_CODE_id = TRAN_CHARGE_CODES['CHARGE_CODE'].unique().tolist()
# FILTER_CODES = '|'.join(CHARGE_CODE_id)
# param = {
#             'CRG_CODE': FILTER_CODES
#         }

In [82]:
# # selecting the transaction code that should be charged
# query = "select id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME from bnk_charge$types where id REGEXP %(CRG_CODE)s"
# query_his = "select id,FLAT_AMOUNT,CALC_TYPE,PERCENTAGE,UPTO_AMOUNT,MINIMUM_AMOUNT,MAXIMUM_AMOUNT,DATE_TIME from bnk_charge$types$history where id REGEXP %(CRG_CODE)s"
# TRAN_CHARGE_HISTORY = mysql_readTable(query,param)
# TRAN_CHARGE_HISTORY_1 = mysql_readTable(query_his,param)
# TRAN_CHARGE_HISTORY = pd.concat([TRAN_CHARGE_HISTORY,TRAN_CHARGE_HISTORY_1], ignore_index=True)
# TRAN_CHARGE_HISTORY = TRAN_CHARGE_HISTORY.sort_values(by='DATE_TIME', ascending=True)
# TRAN_CHARGE_HISTORY.reset_index(drop=True, inplace=True)
# TRAN_CHARGE_HISTORY['TRANSACTION_CODE'] = TRAN_CODE
# TRAN_CHARGE_HISTORY['TAX'] = TRAN_CHARGE_CODES['TAX'][0]
# TRAN_CHARGE_HISTORY.head()

In [83]:
# TRAN_CHARGE_HISTORY['START_DATE'] = np.nan
# TRAN_CHARGE_HISTORY['END_DATE'] = np.nan
# for i in range (0,len(TRAN_CHARGE_HISTORY)):
#     if (i ==0):
#         TRAN_CHARGE_HISTORY['END_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i]
#     elif (TRAN_CHARGE_HISTORY['DATE_TIME'][i] == None):
#         TRAN_CHARGE_HISTORY['START_DATE'][i] = str(pd.to_datetime(TRAN_CHARGE_HISTORY['DATE_TIME']).max())
#     else:
#         TRAN_CHARGE_HISTORY['START_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i - 1]
#         TRAN_CHARGE_HISTORY['END_DATE'][i] = TRAN_CHARGE_HISTORY['DATE_TIME'][i]
# TRAN_CHARGE_HISTORY = TRAN_CHARGE_HISTORY[['id','FLAT_AMOUNT','CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','TRANSACTION_CODE','TAX','START_DATE','END_DATE']]
# TRAN_CHARGE_HISTORY.head()

In [84]:
# df_charges = TRAN_CHARGE_HISTORY
# df_charges['START_DATE'] = pd.to_datetime(df_charges['START_DATE'])
# df_charges['END_DATE'] = pd.to_datetime(df_charges['END_DATE'])
# df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'])

# df_charges.info()

In [85]:

# if(df_charges['FLAT_AMOUNT'][0] == None):
#     cols_to_df_list = ['CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','id','TAX']
    
#     df_chg = pd.DataFrame(columns = cols_to_df_list)
#     for j in range (0,len(df_charges)):
#         df_len = len(df_chg)
#         for i in cols_to_df_list:
#             if df_charges[i][j]:
#                 split_list = fn_str_split(df_charges[i][j])
#                 for k in range (0,len(split_list)):
#                     if((i == "id") | (i == 'TAX') | (i == 'START_DATE') | (i == 'END_DATE') ):
#                         df_chg.loc[df_len + k,i] = df_charges[i][j]
#                     else:
#                         df_chg.loc[df_len + k,i] = split_list[k]
    
    
#     df_chg['id'] = df_chg[['id']].ffill()
#     df_chg['TAX'] = df_chg.groupby('id')['TAX'].ffill()
#     df_chg=df_chg.replace(r'', 0, regex=True)
    
    
    
#     df_chg['PERCENTAGE'] = df_chg['PERCENTAGE'].astype('float')
#     df_chg['MAX_AMT'] = df_chg['UPTO_AMOUNT'].astype('float')
#     df_chg['MIN_CHARGE'] = df_chg['MINIMUM_AMOUNT'].astype('float')
#     df_chg['MAX_CHARGE'] = df_chg['MAXIMUM_AMOUNT'].astype('float')
#     df_chg['TAX'] = df_chg['TAX'].astype('float')
    
    
#     df_chg['MIN_AMT'] = 0
#     for i in range(0,len(df_chg)):
#         if(i == 0):
#             df_chg['MIN_AMT'][i] =  0
#         else:
#             df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1

#     df_chg.loc[(df_chg['MIN_AMT'].isnull()) | (df_chg['MIN_AMT'] > df_chg['MAX_AMT']),'MIN_AMT'] = 0
#     df_chg = df_chg[["id","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE","TAX"]]

# df_chg['START_DATE'] = df_chg.id.map(df_charges.set_index('id')['START_DATE'])
# df_chg['END_DATE'] = df_chg.id.map(df_charges.set_index('id')['END_DATE'])

# df_chg

In [86]:
# df_chg.info()

In [87]:
# # For FLAT AMOUNT with history
# for i in range(0,len(df_charges)):
#     if(df_charges['FLAT_AMOUNT'][i] != None):
#         # Creating expected charge and expected tax percentage column
#         if(pd.isnull(df_charges['START_DATE'][i])):
#             df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
#             df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
#         elif(pd.isnull(df_charges['END_DATE'][i])):
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
#         else:
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['FLAT_AMOUNT'][i]
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
#         #conversion
#         df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
#         df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
#         # expected tax calculation
#         df['EXPECTED_TAX'] = df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX']
    
    
#     elif((df_charges['CALC_TYPE'][i] == 'LEVEL') | (df_charges['CALC_TYPE'][i] == 'BAND')): 
#         if(pd.isnull(df_charges['START_DATE'][i])):
#             df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
#             df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
#         elif(pd.isnull(df_charges['END_DATE'][i])):
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i]
#         else:
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_CHARGE'] = df_charges['PERCENTAGE'][i]
#             df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]),'EXPECTED_TAX'] = df_charges['TAX'][i] 
#         #conversion
#         df['EXPECTED_CHARGE'] = df['EXPECTED_CHARGE'].astype('float')
#         df['EXPECTED_TAX'] = df['EXPECTED_TAX'].astype('float')
#         df_charges['MINIMUM_AMOUNT'] = df_charges['MINIMUM_AMOUNT'].astype('float')
#         df_charges['MAXIMUM_AMOUNT'] = df_charges['MAXIMUM_AMOUNT'].astype('float')
#         # expected charge and tax calculation
#         df['EXPECTED_CHARGE'] = round(df['TRANSACTION_AMOUNT'] /100 * df['EXPECTED_CHARGE'],2)
#         df['EXPECTED_TAX'] = round(df['EXPECTED_CHARGE'] /100 * df['EXPECTED_TAX'],2)
#         # comparing charges with minimum maximum amount
#         if(pd.isnull(df_charges['START_DATE'][i])):
#             # for charge lesser then minimum charge amount
#             if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
#                 df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
#             # for charge greter then minimum charge amount
#             if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
#                 df.loc[(df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
#         elif(pd.isnull(df_charges['END_DATE'][i])):
#             # for charge lesser then minimum charge amount
#             if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
#                 df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
#             # for charge greter then minimum charge amount
#             if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
#                 df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]
#         else:
#             # for charge lesser then minimum charge amount
#             if(pd.isnull(df_charges.iloc[i]['MINIMUM_AMOUNT']) == False):
#                 df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] < df_charges['MINIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MINIMUM_AMOUNT'][i]
#             # for charge greter then minimum charge amount
#             if(pd.isnull(df_charges.iloc[i]['MAXIMUM_AMOUNT']) == False):
#                 df.loc[(df['TIME_STAMP'] >= df_charges['START_DATE'][i]) & (df['TIME_STAMP'] <= df_charges['END_DATE'][i]) & (df['EXPECTED_CHARGE'] > df_charges['MAXIMUM_AMOUNT'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAXIMUM_AMOUNT'][i]

In [88]:
# # for level charge with no hostory
# i = 0
# if(df_charges['FLAT_AMOUNT'][i] == None):
#     if(df_chg['CALC_TYPE'][0] == "LEVEL"):
#         # for expected charge initialization
#         df['EXPECTED_CHARGE'] = 0
#         df['EXPECTED_TAX'] = 0

#         for i in range (0,len(df_chg)):
#             if(pd.isnull(df_chg['START_DATE'][i])):
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for charge lesser then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#                 # for charge greter then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
                
#             elif(pd.isnull(df_chg['END_DATE'][i])):  
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for charge lesser then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#                 # for charge greter then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax & 
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

                    
                    
#             else:
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for charge lesser then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#                 # for charge greter then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

                

#         df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
#         df_chg.reset_index(drop=True, inplace=True)

#         for i in range (0,len(df_chg)):
#             if(pd.isnull(df_chg['START_DATE'][i])):
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for charge lesser then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#                 # for charge greter then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  
            
            
#             elif(pd.isnull(df_chg['END_DATE'][i])):
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for charge lesser then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#                 # for charge greter then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  
            
            
#             else:
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for charge lesser then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#                 # for charge greter then minimum charge amount
#                 if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)  

In [89]:
# # for band charge with hostory
# i = 0
# if(df_charges['FLAT_AMOUNT'][i] == None):
#     if(df_chg['CALC_TYPE'][0] == "BAND"): 
#         # for expected charge initialization
#         df['EXPECTED_CHARGE'] = 0
#         df['EXPECTED_TAX'] = 0

#         for i in range (0,len(df_chg)):
#             if(pd.isnull(df_chg['START_DATE'][i])):
#                 # Chargable amount calculation
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#                 if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
#                 else:
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
            
#             elif(pd.isnull(df_chg['END_DATE'][i])):
#                 # Chargable amount calculation
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#                 if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
#                 else:
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)
                
            
#             else:
#                 # Chargable amount calculation
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#                 if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
#                 else:
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#                 # for charge as per percentage
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
#                 # for tax as per percentage
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df['CHARGE'] * df_chg['TAX'][i] /100),2)

#                 # for expected charge
#                 df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df['EXPECTED_CHARGE'] + df['CHARGE']),2)
#                 # for expected tax
#                 if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#                     df.loc[(df['TIME_STAMP'] >= df_chg['START_DATE'][i]) & (df['TIME_STAMP'] <= df_chg['END_DATE'][i]) & (df['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df['EXPECTED_TAX'] + df['TAX']),2)

In [90]:
# Need's to handle
#--------------------
# LEVEL
# BAND
# MULTIPLE LEVELS/MULTIPLE BANDS
# DIFFERENT CURRENCY
# DIFFERENT CURRENCY WITH LEVELS/BANDS
# PERFORMANCE
#-----------------
# MONTH WISE EXECUTION
# DAILY DATA STORAGE IN IN DB 

In [91]:
# import numpy as np
# from itertools import chain

In [92]:
# # selecting the transaction code charges
# mysql_connection = mysql.connector.connect(**config)
# #df_charges = pd.read_sql('select tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MAXIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
# df_charges = pd.read_sql('select * from  bnk_charge$types where id ="TEST"', con=mysql_connection)
# mysql_connection.close()
# df_charges.head()

In [93]:
# # return list from series of comma-separated strings
# def chainer(s):
#     return list(chain.from_iterable(s.str.split('^')))

In [94]:
# # calculate lengths of splits
# lens = df_charges['CALC_TYPE'].str.split('^').map(len)

# # create new dataframe, repeating or chaining as appropriate
# df_charges = pd.DataFrame({'TRANSACTION_CODE': np.repeat(df_charges['id'], lens),
#                     'CALCULATION_BASIS':np.repeat(df_charges['CALCULATION_BASIS'],lens),
#                     'CALC_TYPE': chainer(df_charges['CALC_TYPE']),
#                     'PERCENTAGE': chainer(df_charges['PERCENTAGE']),
#                     'MAX_AMT': chainer(df_charges['UPTO_AMOUNT']),
#                     'MIN_CHARGE': chainer(df_charges['MINIMUM_AMOUNT']),
#                     'MAX_CHARGE': chainer(df_charges['MAXIMUM_AMOUNT'])})

# df_charges.reset_index(drop=True, inplace=True)
# df_charges

In [95]:
# df_charges['PERCENTAGE'] = df_charges['PERCENTAGE'].astype('float')
# df_charges['MAX_AMT'] = df_charges['MAX_AMT'].astype('int')
# df_charges['MIN_CHARGE'] = df_charges['MIN_CHARGE'].astype('int')
# df_charges['MAX_CHARGE'] = df_charges['MAX_CHARGE'].astype('int')

In [96]:
# df_charges['MIN_AMT'] = 0
# for i in range(0,len(df_charges)):
#     if(i == 0):
#         df_charges['MIN_AMT'][i] =  0
#     else:
#         df_charges['MIN_AMT'][i] =  df_charges['MAX_AMT'][i-1] + 1

# df_charges = df_charges[["TRANSACTION_CODE","CALCULATION_BASIS","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE"]]
# df_charges

In [97]:
# # fdr level calculations
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [98]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0

# for i in range (0,len(df_charges)):
    
#     # for expected charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_charges['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['TRANSACTION_AMOUNT'] * df_charges['PERCENTAGE'][i] /100),2)
#     # for expected charge lesser then minimum charge amount
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_charges['MAX_AMT'][i]) & (df1['EXPECTED_CHARGE'] < df_charges['MIN_CHARGE'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MIN_CHARGE'][i]
#     # for expected charge greter then minimum charge amount
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_charges['MAX_AMT'][i]) & (df1['EXPECTED_CHARGE'] > df_charges['MAX_CHARGE'][i]) , 'EXPECTED_CHARGE'] =  df_charges['MAX_CHARGE'][i]

# # df_charges.loc[df_charges['MAX_AMT'].idxmax()]
# # for mapping percentage of charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT),'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['TRANSACTION_AMOUNT'] * df_charges.loc[df_charges['MAX_AMT'].idxmax()].PERCENTAGE /100),2)
# # for mapping min charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] < df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE
# # for mapping max charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE
# df1.head()

In [99]:
# # for band calculations
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [100]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0

# for i in range (0,len(df_charges)):
#     # Chargable amount calculation
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - (df_charges['MIN_AMT'][i])
#     df1.loc[(df1['CHARGABLE_AMT'] > (df_charges['MAX_AMT'][i] - df_charges['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_charges['MAX_AMT'][i] - df_charges['MIN_AMT'][i]
#     # for expected charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_charges['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['CHARGABLE_AMT'] * df_charges['PERCENTAGE'][i] /100),2)

# # for mapping percentage of charge, greater then last upto amount
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT),'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT
# # for expected charge as per last percentage
# df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT), 'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['CHARGABLE_AMT'] * df_charges.loc[df_charges['MAX_AMT'].idxmax()].PERCENTAGE /100),2)
# df1.head()

In [101]:
# def fn_str_split(string):
#     return string.split('^')

In [102]:
# # selecting the transaction code charges
# mysql_connection = mysql.connector.connect(**config)
# #df_charges = pd.read_sql('select tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MAXIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from bnk_charge$types bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
# df_charges = pd.read_sql('select bct.id,tcc.TRANSACTION_CODE,bct.CALCULATION_BASIS,bct.FLAT_AMOUNT,bct.CALC_TYPE,bct.PERCENTAGE,bct.UPTO_AMOUNT,bct.MINIMUM_AMOUNT,bct.MAXIMUM_AMOUNT,tcc.TAX from ft$commission$type bct inner join transaction$charge$codes tcc on bct.id = tcc.CHARGE_CODE where tcc.TRANSACTION_CODE = 738', con=mysql_connection)
# mysql_connection.close()
# df_charges.head()

In [103]:
# cols_to_df_list = ['CALC_TYPE','PERCENTAGE','UPTO_AMOUNT','MINIMUM_AMOUNT','MAXIMUM_AMOUNT','id','TAX']
# df_chg = pd.DataFrame(columns = cols_to_df_list)
# for j in range (0,len(df_charges)):
#     df_len = len(df_chg)
#     for i in cols_to_df_list:
#         if df_charges[i][j]:
#             split_list = fn_str_split(df_charges[i][j])
#             for k in range (0,len(split_list)):
#                 if((i == "id") | (i == 'TAX')):
#                     df_chg.loc[df_len + k,i] = df_charges[i][j]
#                 else:
#                     df_chg.loc[df_len + k,i] = split_list[k]
# df_chg

In [104]:
# df_chg['id'] = df_chg[['id']].ffill()
# df_chg['TAX'] = df_chg.groupby('id')['TAX'].ffill()
# df_chg

In [105]:
# df_chg['PERCENTAGE'] = df_chg['PERCENTAGE'].astype('float')
# df_chg['MAX_AMT'] = df_chg['UPTO_AMOUNT'].astype('float')
# df_chg['MIN_CHARGE'] = df_chg['MINIMUM_AMOUNT'].astype('float')
# df_chg['MAX_CHARGE'] = df_chg['MAXIMUM_AMOUNT'].astype('float')
# df_chg['TAX'] = df_chg['TAX'].astype('float')
# df_chg

In [106]:
# df_chg['MIN_AMT'] = 0
# for i in range(0,len(df_chg)):
#     if(i == 0):
#         df_chg['MIN_AMT'][i] =  0
#     else:
#         df_chg['MIN_AMT'][i] =  df_chg['MAX_AMT'][i-1] + 1

# df_chg.loc[(df_chg['MIN_AMT'].isnull()) | (df_chg['MIN_AMT'] > df_chg['MAX_AMT']),'MIN_AMT'] = 0
# df_chg = df_chg[["id","CALC_TYPE","PERCENTAGE","MIN_AMT","MAX_AMT","MIN_CHARGE","MAX_CHARGE","TAX"]]
# df_chg

In [107]:
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [108]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0
# df1['EXPECTED_TAX'] = 0

# for i in range (0,len(df_chg)):
#     # for charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'CHARGE'] = round((df1['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#     # for charge lesser then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df1['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#     # for charge greter then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]) & (df1['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#     # for tax as per percentage
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'TAX'] = round((df1['CHARGE'] * df_chg['TAX'][i] /100),2)
    
#     # for expected charge
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_CHARGE'] = round((df1['EXPECTED_CHARGE'] + df1['CHARGE']),2)
#     # for expected tax
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['TRANSACTION_AMOUNT'] <= df_chg['MAX_AMT'][i]), 'EXPECTED_TAX'] = round((df1['EXPECTED_TAX'] + df1['TAX']),2)
    
# # # df_charges.loc[df_charges['MAX_AMT'].idxmax()]
# # # for mapping percentage of charge, greater then last upto amount
# # df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT),'EXPECTED_CHARGE'] = round(df1['EXPECTED_CHARGE'] + (df1['TRANSACTION_AMOUNT'] * df_charges.loc[df_charges['MAX_AMT'].idxmax()].PERCENTAGE /100),2)
# # # for mapping min charge, greater then last upto amount
# # df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] < df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MIN_CHARGE
# # # for mapping max charge, greater then last upto amount
# # df1.loc[(df1['TRANSACTION_AMOUNT'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_AMT) & (df1['EXPECTED_CHARGE'] > df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE),'EXPECTED_CHARGE'] = df_charges.loc[df_charges['MAX_AMT'].idxmax()].MAX_CHARGE
# df1.tail()

In [109]:
# df_chg = df_chg.loc[(df_chg['MAX_AMT'].isnull())]
# df_chg.reset_index(drop=True, inplace=True)
# df_chg.head()

In [110]:
# for i in range (0,len(df_chg)):
#     # for charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df1['TRANSACTION_AMOUNT'] * df_chg['PERCENTAGE'][i] /100),2)
#     # for charge lesser then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MIN_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['CHARGE'] < df_chg['MIN_CHARGE'][i]) , 'CHARGE'] =  df_chg['MIN_CHARGE'][i]
#     # for charge greter then minimum charge amount
#     if(pd.isnull(df_chg.iloc[i]['MAX_CHARGE']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]) & (df1['CHARGE'] > df_chg['MAX_CHARGE'][i]) , 'CHARGE'] =  df_chg['MAX_CHARGE'][i]
#     # for tax as per percentage
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df1['CHARGE'] * df_chg['TAX'][i] /100),2)
    
#     # for expected charge
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df1['EXPECTED_CHARGE'] + df1['CHARGE']),2)
#     # for expected tax
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df1['EXPECTED_TAX'] + df1['TAX']),2)
# df1.tail()   

In [111]:
# df1 = pd.read_csv("sample.csv")
# df1.head()

In [112]:
# # for expected charge initialization
# df1['EXPECTED_CHARGE'] = 0
# df1['EXPECTED_TAX'] = 0

# for i in range (0,len(df_chg)):
#     # Chargable amount calculation
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#     if(pd.isnull(df_chg.iloc[i]['MAX_AMT']) == False):
#         df1.loc[(df1['CHARGABLE_AMT'] > (df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df_chg['MAX_AMT'][i] - df_chg['MIN_AMT'][i]
#     else:
#         df1.loc[(df1['TRANSACTION_AMOUNT'] > (df_chg['MIN_AMT'][i])), 'CHARGABLE_AMT'] = df1['TRANSACTION_AMOUNT'] - (df_chg['MIN_AMT'][i])
#     # for charge as per percentage
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'CHARGE'] = round((df1['CHARGABLE_AMT'] * df_chg['PERCENTAGE'][i] /100),2)
#     # for tax as per percentage
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'TAX'] = round((df1['CHARGE'] * df_chg['TAX'][i] /100),2)

#     # for expected charge
#     df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_CHARGE'] = round((df1['EXPECTED_CHARGE'] + df1['CHARGE']),2)
#     # for expected tax
#     if(pd.isnull(df_chg.iloc[i]['TAX']) == False):
#         df1.loc[(df1['TRANSACTION_AMOUNT'] >= df_chg['MIN_AMT'][i]), 'EXPECTED_TAX'] = round((df1['EXPECTED_TAX'] + df1['TAX']),2)
# df1.tail()